# Bag of Words

## Term Document Frequency

![](images/dtm.PNG)

## TF- IDF Term Frequency Inverse Document Frequency

Words that appear in many documents are probably less meaningful

 We are interested in obtaining VXD type matrix where V is the vocabulary size and D is the Vector Dimensionality
 
 ![](images/vxd.PNG)

# Word Embeddings

Feature vector representing a word

## Word Analogies

There is no concept of word analogies with algorithms like word2vec and Glove.They just suddenly emerge out of the model and training process.

![](images/word_analogies.PNG)

## Get Word Similarity

## Similarity Metric

1. Euclidean
2. Cosine

# Glove Analogies

Get glove.6B.50d file for word embeddings

In [6]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
#Euclidean
def dist1(a, b):
    return np.linalg.norm(a - b)
#Cosine
def dist2(a, b):
    return 1 - a.dot(b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [8]:
# pick a distance type
dist, metric = dist2, 'cosine'

In [9]:
def find_analogies(w1, w2, w3):
  for w in (w1, w2, w3):
    if w not in word2vec:
      print("%s not in dictionary" % w)
      return

  king = word2vec[w1]
  man = word2vec[w2]
  woman = word2vec[w3]
  v0 = king - man + woman

  distances = pairwise_distances(v0.reshape(1, D), embedding, metric=metric).reshape(V)
  idxs = distances.argsort()[:4]
  for idx in idxs:
    word = idx2word[idx]
    if word not in (w1, w2, w3): 
      best_word = word
      break

  print(w1, "-", w2, "=", best_word, "-", w3)

In [10]:
def nearest_neighbors(w, n=5):
  if w not in word2vec:
    print("%s not in dictionary:" % w)
    return

  v = word2vec[w]
  distances = pairwise_distances(v.reshape(1, D), embedding, metric=metric).reshape(V)
  idxs = distances.argsort()[1:n+1]
  print("neighbors of: %s" % w)
  for idx in idxs:
    print("\t%s" % idx2word[idx])

In [6]:
print('Loading word vectors...')
word2vec = {}
embedding = []
idx2word = []
with open('glove.6B.50d.txt', encoding='utf-8') as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
    embedding.append(vec)
    idx2word.append(word)
print('Found %s word vectors.' % len(word2vec))
embedding = np.array(embedding)
V, D = embedding.shape

Loading word vectors...
Found 400000 word vectors.


In [7]:
find_analogies('king', 'man', 'woman')

king - man = queen - woman


In [8]:
find_analogies('france', 'paris', 'london')

france - paris = britain - london


# Word2Vec Analogies

Get GoogleNews-vectors-negative300.bin file for word embedding

In [11]:
from gensim.models import KeyedVectors

In [12]:
word_vectors = KeyedVectors.load_word2vec_format(
  'GoogleNews-vectors-negative300.bin',
  binary=True
)

In [13]:
def find_analogies(w1, w2, w3):
  r = word_vectors.most_similar(positive=[w1, w3], negative=[w2])
  print("%s - %s = %s - %s" % (w1, w2, r[0][0], w3))

def nearest_neighbors(w):
  r = word_vectors.most_similar(positive=[w])
  print("neighbors of: %s" % w)
  for word, score in r:
    print("\t%s" % word)


# Word Embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So, the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

To solve this problem and greatly increase the efficiency of our networks, we use what are called **embeddings**. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![](images/embedding.PNG)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.


## Word2Vec

[Word2Vec](https://arxiv.org/pdf/1310.4546.pdf)
[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)

Word2Vec is one of the most popular technique to learn word embeddings using shallow neural network.

We’re going to train a simple neural network with a single hidden layer to perform a certain task, but then we’re not actually going to use that neural network for the task we trained it on! Instead, the goal is actually just to learn the weights of the hidden layer–we’ll see that these weights are actually the “word vectors” that we’re trying to learn.

Different model architectures that can be used with Word2Vec 

1. CBOW Neural Network Model
2. Skipgram Neural Network Model

![](images/cbow_skipgram.PNG)


Different ways to train Word2vec Model:
1. Heirarchial Softmax
2. Negative Sampling

### Skipgram

![](images/skipgram_g.PNG)


**eg: The quick brown fox jumps over the lazy dog**
![](images/skipgram.PNG)


We’re going to train the neural network to do the following. Given a specific word in the middle of a sentence (the input word), look at the words **nearby** and pick one at random. The network is going to tell us the probability for every word in our vocabulary of being the “nearby word” that we chose.

**nearby** - there is actually a "window size" parameter to the algorithm. A typical window size might be 5, meaning 5 words behind and 5 words ahead (10 in total).

We’ll train the neural network to do this by feeding it word pairs found in our training documents.
The network is going to learn the statistics from the number of times each pairing shows up. So, for example, the network is probably going to get many more training samples of (“Soviet”, “Union”) than it is of (“Soviet”, “Sasquatch”). When the training is finished, if you give it the word “Soviet” as input, then it will output a much higher probability for “Union” or “Russia” than it will for “Sasquatch”.

#### Skipgram Model Details

Skipgram architecture consists of:
1. [Embedding layer / Hidden Layer](https://pytorch.org/docs/stable/nn.html#embedding)

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension.(300 features is what Google used in their published model trained on the Google news dataset (you can download it from here)



2. Softmax Output Layer<br>
The output layer will have output neuron (one per word in our vocabulary) will produce an output between 0 and 1, and the sum of all these output values will add up to 1.


![](images/skip_gram_arch.PNG)

**Working**
1. The input words are passed in as batches of input word tokens. 
2. This will go into a hidden layer of linear units (our embedding layer). 
3. Then, finally into a softmax output layer. We'll use the softmax layer to make a prediction about the context words by sampling, as usual.

### Training Word2Vec Model

Word2Vec network is a huge network in terms of parameters.
Say we had word vectors with 300 components, and a vocabulary of 10,000 words. Recall that the neural network had two weight matrices–a hidden layer and output layer. Both of these layers would have a weight matrix with 300 x 10,000 = 3 million weights each!

Hence to reduce compute burden of the training process,the research paper proposed following two innovations:
1. **Subsampling frequent words** to decrease the number of training examples.
2. Modifying the optimization objective with a technique they called **Negative Sampling**,instead of normal cross entropy which causes each training sample to update only a small percentage of the model’s weights which makes training the network very inefficient.

#### Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

#### Negative Sampling

As discussed above,the the skip-gram neural network has a tremendous number of weights, all of which would be updated slightly by every one of our millions or billions of training samples!

Negative sampling addresses this by having each training sample only modify a small percentage of the weights, rather than all of them.

With negative sampling, we are instead going to randomly select just a small number of “negative” words (let’s say 5) to update the weights for. (In this context, a “negative” word is one for which we want the network to output a 0 for). We will also still update the weights for our “positive” word (which is the word “quick” in our current example).

***The paper says that selecting 5-20 words works well for smaller datasets, and you can get away with only 2-5 words for large datasets.***



#### Selecting Negative Samples

The “negative samples” (that is, the 5 output words that we’ll train to output 0) are selected using a “unigram distribution”, where more frequent words are more likely to be selected as negative samples.

**Modification in Cost Function**

We change the loss function to only care about correct example and a small amount of wrong examples.
![](https://render.githubusercontent.com/render/math?math=-%20%5Clarge%20%5Clog%7B%5Csigma%5Cleft%28u_%7Bw_O%7D%5Chspace%7B0.001em%7D%5E%5Ctop%20v_%7Bw_I%7D%5Cright%29%7D%20-%0A%5Csum_i%5EN%20%5Cmathbb%7BE%7D_%7Bw_i%20%5Csim%20P_n%28w%29%7D%5Clog%7B%5Csigma%5Cleft%28-u_%7Bw_i%7D%5Chspace%7B0.001em%7D%5E%5Ctop%20v_%7Bw_I%7D%5Cright%29%7D&mode=display)

First part of the Loss function:
![](https://render.githubusercontent.com/render/math?math=%5Clarge%20%5Clog%7B%5Csigma%5Cleft%28u_%7Bw_O%7D%5Chspace%7B0.001em%7D%5E%5Ctop%20v_%7Bw_I%7D%5Cright%29%7D&mode=display)
we take the log-sigmoid of the inner product of the output word vector  and the input word vector.

Second part of the Loss function:


let's first look at 

$$\large \sum_i^N \mathbb{E}_{w_i \sim P_n(w)}$$ 

This means we're going to take a sum over words $w_i$ drawn from a noise distribution $w_i \sim P_n(w)$. The noise distribution is basically our vocabulary of words that aren't in the context of our input word. In effect, we can randomly sample words from our vocabulary to get these words. $P_n(w)$ is an arbitrary probability distribution though, which means we get to decide how to weight the words that we're sampling. This could be a uniform distribution, where we sample all words with equal probability. Or it could be according to the frequency that each word shows up in our text corpus, the unigram distribution $U(w)$. The authors found the best distribution to be $U(w)^{3/4}$, empirically. The power makes less frequent words be sampled more often

Finally, in 

$$\large \log{\sigma\left(-u_{w_i}\hspace{0.001em}^\top v_{w_I}\right)},$$ 

we take the log-sigmoid of the negated inner product of a noise vector with the input vector. 

![](images/neg_sampling_loss.PNG)

To give you an intuition for what we're doing here, remember that the sigmoid function returns a probability between 0 and 1. The first term in the loss pushes the probability that our network will predict the correct word $w_O$ towards 1. In the second term, since we are negating the sigmoid input, we're pushing the probabilities of the noise words towards 0.

### Word2Vec Implementation codes

1. Numpy implementation
2. Pytorch Implementation

## Global Vector for Word Representation(GloVe)

[GLoVe](https://www.aclweb.org/anthology/D14-1162)

**Motivation**

The Word2Vec -***context window-based methods*** suffer from the disadvantage of not learning from the global corpus statistics. As a result, repetition and large-scale patterns may not be learned as well with these models.

This method combines elements from the two main word embedding models which existed when GloVe was proposed: 
1. Count based approach - Global matrix factorization 
2. Direct Prediction - Local context window methods


![](images/count_based_vs_direct_pred.PNG)

### Global Matrix Factorization - Count based approach

It is the process of using matrix factorization methods from linear algebra to perform rank reduction on a large term-frequency matrix.
These matrices can be represented as :
1. Term-Document Frequency - rows are words and the columns are documents (or sometimes paragraphs)
2. Term-Term Frequencies   - words on both axes and measure co-occurrence

**Latent semantic analysis (LSA)**
It is a technique in natural language processing for analyzing relationships between a set of documents and the terms they contain by applying **Global matrix factorization** to term-document frequency matrices.
In **LSA** the high-dimensional matrix is reduced via **singular value decomposition (SVD).**

![](images/svd.jpg)

### Local Context Window Direct prediction

1. **Skip-gram model**<br>
By passing a window over the corpus line-by-line and learning to predict either the surroundings of a given word  
2. **Continuous Bag of Words Model (CBOW)**<br>
Predict a word given its surroundings.
Note the bag-of-words problem is often shortened to “CBOW”.

**Skip-gram**: works well with small amount of the training data, represents well even rare words or phrases.<br>
**CBOW**: several times faster to train than the skip-gram, slightly better accuracy for the frequent words.


![](images/cbow_skipgram.PNG)


### GloVe embedding generation algorithm 

GloVe technique improves on these previous methods by making changes in the following:

1. **Co-occurance Probabilities**<br>
Instead of learning the raw co-occurrence probabilities, it may make more sense to learn ratios of these co-occurrence probabilities, which seem to better discriminate subtleties in term-term relevance.

To illustrate this, we borrow an example from their paper: suppose we wish to study the relationship between two words, i = ice and j = steam. We’ll do this by examining the co-occurrence probabilities of these words with various “probe” words.

Co-occurrence probability of an arbitrary word i with an arbitrary word j to be the probability that word j appears in the context of word i.
![](images/co-occurance_matrix.PNG)
***X_i*** is defined as the number of times any word appears in the context of word i, so it’s defined as the sum over all words k of the number of times word k occurs in the context of word i.

Let us take few probe words and see how does the ratio appears:
1. If we choose a probe word k = solid which is closely related to i = ice but not to j = steam, we expect the ratio P_{ik}/P_{jk} of co-occurrence probabilities to be large
2. If we choose a probe word k = gas we would expect the same ratio to be small, since steam is more closely related to gas than ice is.
3. If we choose a probe word k = water , which are closely related to both ice and steam, but not more to one than the other ,we expect our ratio to be close to 1 since there shouldn’t be any bias to one of ice or steam
4. If we choose a probe word k = fashion ,which are not closely related to either of the words in question, we expect our ratio to be close to 1 since there shouldn’t be any bias to one of ice or steam

![](images/co-occurance_ex.PNG)

Noting that the ratio P<sub>ik</sub> /P<sub>jk</sub> depends on three words i, j, and k,
the most general model takes the form,
![](images/naive_form.PNG)

***In this equation, the right-hand side is extracted from the corpus, and F may depend on some as-of-yet unspecified parameters. The number of possibilities for F is vast, but by enforcing a few desiderata we can select a
unique choice***.
We have two word vectors which we’d like to discriminate between, and a context word vector which is used to this effect.So to encode information about the ratios between two words, the authors suggest using vector differences as inputs to our function
**Vector Difference Model**
![](images/vector_difference_model.PNG)

So now,vector difference of the two words **i** and **j** we’re comparing as an input instead of both of these words individually, since our output is a ratio between their co-occurrence probabilities with the context word.

Now we have two arguments, the context word vector, and the vector difference of the two words we’re comparing. Since the authors wish to take scalar values to scalar values (note the ratio of probabilities is a scalar), the dot product of these two arguments is taken

![](images/scalar_input_model.PNG)

Next, note that for word-word co-occurrence matrices, the distinction between a word and a context word is arbitrary and that we are free to exchange the two roles.Our final model should be invariant under this relabeling, but above equation is not.

After applying Homomorphism condition:
![](images/homomorphism.PNG)

we arrive at the equation as in the paper
![](images/glove_homo.PNG)

# Playground

In [15]:
import numpy as np

In [16]:
def logit(x):
    return np.log(x/(1-x))

def sigmoid(x):
    return 1/ (1+ np.exp(-x))

In [17]:
# p(y=1/x)= Sigmoid(logit(p))
sigmoid(logit(0.8))

0.8

In [18]:
#p(y=0/x) = sigmoid(-logit(p))
sigmoid(-logit(0.8))

0.19999999999999996

In [19]:
# p(y=1/x)= Sigmoid(logit(p))  , p(y=0/x) = sigmoid(-logit(p))
# sigmoid(logit) + sigmoid(-logit) =1
#Inverse of logit is sigmoid i.e p = sigmoid(logit(p))

In [ ]:
# sigmoid(-x)= 1-sigmoid(x)

In [20]:
print(sigmoid(logit(0.8)))
print(sigmoid(-logit(0.8)))
print(sigmoid(logit(0.8))+sigmoid(-logit(0.8)))

0.8
0.19999999999999996
1.0


In [21]:
print(np.log(sigmoid(0.9)))
print(np.log(sigmoid(-0.1)))
print(-(np.log(sigmoid(0.8))+np.log(sigmoid(-0.1))))

-0.3411538747320879
-0.744396660073571
1.1154973260213485


# Reference

1. [Bayes Rule](https://math.stackexchange.com/questions/1037327/extended-bayes-theorem-pa-b-c-d-constructing-a-bayesian-network)
2. [Word2Vec from Chris McCormick ](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
3. [Word Embedding](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/)
4. [Word2Vec](https://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html)
5. [First Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
6. [Video: Intuition & Use-Cases of Embeddings in NLP & beyond](http://jalammar.github.io/)
7. [Neural Information Processing Systems, paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for Word2Vec also from Mikolov et al.
8. [Skipgram-Pytorch](https://github.com/udacity/deep-learning-v2-pytorch/blob/master/word2vec-embeddings/Negative_Sampling_Solution.ipynb)
9. [Word2Vec-Pytorch](https://github.com/dthiagarajan/word2vec-pytorch)
10. [GloVe](https://towardsdatascience.com/emnlp-what-is-glove-part-iv-e605a4c407c8)
11. [Glove](https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010)